In [38]:
import tensorflow as tf

import numpy as np
import os
import time
import datetime

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print(idx2char)
print(char2idx)
print( text_as_int[:250])

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 5

In [6]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print( examples_per_epoch)
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

11043
F
i
r
s
t


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [8]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [10]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(list(dataset.take(1).as_numpy_iterator()))

[(array([[ 1, 58, 39, ..., 43,  1, 44],
       [ 1, 46, 53, ..., 57, 58,  1],
       [57,  1, 58, ..., 43,  0, 32],
       ...,
       [32, 53,  1, ..., 56,  0, 20],
       [52, 42,  1, ..., 56,  1, 61],
       [46, 43,  1, ..., 46,  8,  0]]), array([[58, 39, 49, ...,  1, 44, 39],
       [46, 53, 52, ..., 58,  1, 43],
       [ 1, 58, 46, ...,  0, 32, 46],
       ...,
       [53,  1, 25, ...,  0, 20, 53],
       [42,  1, 58, ...,  1, 61, 47],
       [43,  1, 55, ...,  8,  0,  0]]))]


In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [29]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
print( model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)
[17 28 61 57 36 20 28 20 39 25  4 39  1 59 45  1 19  0 39 39 29 29 42 42
 29 33 42 42  2 39 39  3 58 45 39 58 39 39 33 39 34 45 39 39 39 25 56 18
 53  0  7 42 28 47 42  0 42  7 18 50 39 13 13  4 47 39 39 39 28 39 57 45
 39  0 39  2 33 39 39 39 39 28 49 39  2 50 25 20 29 40 34 39 39 39 50 18
 49  0  7 42]
tf.Tensor(
[26 17 10  0 26 53  2  1 61 46 63 12  1  1 35 46 43 52  1 46 43  1 58 46
 39 58  1 47 57  1 51 63  1 46 59 57 40 39 52 42  1 52 53 61  0 15 39 51
 43  1 58 53  1 51 43  6  1 39 57  1 21  1 44 53 50 50 53 61  5 42  1 20
 43 52 56 63  5 57  1 41 53 56 57 43  6  0 35 46 43 52  1 57 41 39 56 41
 43  1 58 46], shape=(100,), dtype=int64)
tf.Tensor(
[ -9  11  51  57  10 -33  26  19 -22 -21 -59  27   0  58  10 -45 -24 -52
  38  -7 -14  28 -16  -4 -10 -25  41  -5 -55  38 -12 -60  57  -1 -20   1
  -1   0 -19  -3  33  -7 -14 -22  39  10  17 -33  10  -1 -51 -11  27  -4
  -1  -6  41 -32 -39  49  18  12 -31 -49  -3 -11 -14 -22  23 

In [34]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [35]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1743264


In [42]:
model.compile( optimizer='adam', loss=loss)
log_dir = "/home/manju/code/ML/src/manju_tensorflow_examples/logs/fit/" \
 + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + 'word_prediction_gru_1024'
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

cp_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS=10
train_flag = False
if train_flag:
    model.fit(dataset, epochs=EPOCHS, callbacks=[tb_callback, cp_callback])

In [51]:
if not train_flag:
    print(tf.train.latest_checkpoint(checkpoint_dir))
    
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

    model.build(tf.TensorShape([1, None]))
    
    print( model.summary())

./training_checkpoints/ckpt_10
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = .10

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    print(predictions.numpy())
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

if not train_flag:
    print(generate_text(model, start_string=u"ROMEO: "))

[[-3.20077438e+01 -1.61929264e+01 -1.64210262e+01  2.67361259e+01
   2.70761147e+01 -1.30076036e+01 -1.81096630e+01 -2.74805775e+01
  -1.28160849e+01  1.82855053e+01  2.63921642e+01 -1.75356388e+01
  -1.92005234e+01  6.31192474e+01  2.62791042e+01  5.99528580e+01
   7.11211014e+01  4.85151443e+01  1.04238100e+01  5.37299042e+01
   1.43736429e+01  6.82252045e+01  2.17447453e+01  6.31804314e+01
   3.74064331e+01  3.14168549e+01  2.82724075e+01  4.99232368e+01
   1.09930677e+01  1.95908165e+01  4.22410250e+00  3.30668678e+01
   3.58026810e+01  5.98908691e+01  2.68137455e+01  4.50763779e+01
   3.08914299e+01  6.12809448e+01  4.30731163e+01 -1.11711655e+01
  -2.73750038e+01 -3.81815052e+00 -2.12617800e-01 -1.92816758e+00
  -2.24518566e+01 -1.53647108e+01 -9.58689117e+00 -4.18366385e+00
   3.34718156e+00  1.47045794e+01 -3.50768623e+01 -2.93447037e+01
  -5.00544815e+01 -5.97375488e+00 -3.53606110e+01  1.21627510e+00
  -7.51043243e+01 -2.70825005e+01 -2.61508102e+01 -1.69689198e+01
  -1.62433

[[-55.37275    -67.59756    -48.702324   -75.873726   -94.659966
  -33.615      -66.191536   -46.795216   -56.986767   -97.72477
  -68.45553    -49.711452   -33.83792     -4.459235     1.1209588
    0.41786775  -9.450946    -3.3421242   -7.068425   -13.504487
  -14.431981   -24.968033   -29.013557   -15.865798    14.774927
   -8.747602   -36.46134    -39.563538   -14.342448   -45.674824
    2.1201413  -19.590082    15.656836   -54.36775     -2.3986707
  -18.420393   -60.25877    -26.514568   -81.153755    25.429123
   35.55151     36.919132    34.237206    38.93811     36.003754
   33.806625    41.197964    25.704763    19.428017    33.94856
   25.824911    30.428236    25.326975    19.24427     28.841476
    9.763684    32.618336    33.888798    30.442776     0.74565023
   23.592855    43.520077   -77.13561     14.594023   -29.23798   ]]
[[  16.668423      5.1432633   -13.431028   -108.99987     -72.954735
     2.7096825     2.6330416   -33.387066      0.640955    -59.65944
     0.978

[[  13.526783    66.72215    -30.420094  -112.24347    -97.203445
    11.218286    19.83551      5.042286     7.4314747  -86.04161
   -11.491009   -11.479422   -23.838575   -51.73502    -67.4032
   -69.11586    -64.85311    -76.10771    -79.89244    -57.393787
   -88.70058    -17.272703   -98.52829    -63.631397   -93.64918
   -76.75012    -70.072075   -52.13122    -71.89048    -74.91356
   -72.807076   -54.571434   -45.82482    -84.90834    -98.53615
   -49.425076  -103.84385    -47.955875   -63.989735     2.7064207
    41.205338    47.970615    23.114374    40.54648     21.518457
    44.27984    -19.126337     9.741755    -7.3938866    3.7470293
    36.86361     40.369915    23.69953     15.66103    -21.442535
   -17.30614     20.386887    43.488216    53.10163     -7.37254
   -22.883396    36.87782    -63.76964     -4.66976    -44.45724  ]]
[[ -96.246155    -99.30433     -92.28779    -103.53618     -85.29464
     4.1292214  -100.49711     -55.341076    -90.41429    -103.71803
   -89

[[147.60336     98.99751     38.073498   -29.310978   -49.08419
   85.32462     18.019194    70.13326     50.8176       0.5641977
   49.046886    24.57682     44.43566     24.032488    12.015466
   15.033587     4.2827773  -31.203598    17.445673     0.47883055
   -5.9878845   19.268847    -6.8730507  -33.2306      -2.892662
    1.074146    32.651714     3.0161083  -19.457851   -16.646591
   15.041894    17.692427     7.1795225    5.643345   -35.66657
    7.718614   -44.841457     5.543743    -3.7327492  -27.288698
  -39.926834   -38.956337   -38.31318     25.725098   -28.917086
  -34.940277   -57.114464   -10.431394   -51.75507    -89.64826
  -13.909718   -65.535164    -1.7261195  -19.498205   -56.737526
  -63.41259    -39.985462     3.1650388  -62.4233     -18.678345
  -38.726154   -33.19408    -79.94577    -70.27699     -0.203562  ]]
[[  91.98127     -38.77569     -55.811058    -52.850025    -94.47739
    29.519901   -100.63619     -12.186208    -67.46055      20.185703
   -64.07466

[[  -3.488418      1.2348492     0.72845703  -90.65495     -64.2227
    12.070552     -5.934829    -35.53596      -6.1150727   -71.75304
   -15.979931    -11.896972      6.6348696   -43.08312     -48.470303
  -114.89046     -92.46457     -38.8354      -46.295494    -71.93431
   -74.165016    -12.715453    -48.737503    -89.28865     -66.89637
   -43.42776     -28.459093    -26.208437    -33.569523    -49.26617
   -69.38386     -98.04984     -66.03946     -54.89596     -46.682823
   -48.569267    -96.77039     -54.209507   -105.61093     111.39266
    22.09307     -55.320812    -69.609604     86.7031        8.16131
   -79.63206     -68.14874      96.21453     -23.9659      -31.1949
    -4.0066376    14.591484      9.624409     98.266335     17.142687
   -35.18687      -1.8817492    15.578751    -48.247208     85.32852
   -32.4475       -2.4689627   -35.320858     46.545246    -57.616947  ]]
[[-26.312479   -17.561182    16.27575    -57.053677   -36.648743
  -52.96108     -7.1313076  -18.

...

[[  12.754889    41.92355    -13.1623745 -130.0472     -80.98602
   -13.554159    14.191842    -1.9504279    5.338023   -83.16171
    -2.8436315   15.463895    -3.8739305  -61.40034    -79.711914
  -106.998405   -82.04706    -57.593994   -63.95707    -69.49014
   -71.67684    -43.647232   -71.73457   -126.72078    -47.341843
   -74.41463    -42.669495   -68.331635   -65.73535    -71.917336
   -91.59601    -69.055664   -51.656796   -75.04766    -82.33585
   -68.22845    -65.39787   -102.064316   -83.70519     65.71728
     4.07507      3.3606284   60.11111     34.347073    42.999153
    46.690407    44.62834     40.171696   -25.732473   -32.996567
    34.52875      4.0694137   54.014088    -1.0127467  -16.146374
    -7.5345464   14.888412    39.242725    53.383823   -42.296204
   -22.314453    32.23715     -2.7120793   25.138056   -22.06953  ]]
[[ -15.615017      0.14470913  -16.400055    -91.61058     -80.77213
   -15.584016     15.322981      8.312578      0.65855604  -74.295746
    -

[[ -11.609961   15.943461  -37.75872   -73.03634   -61.924835    8.998988
   -18.43122   -17.06708   -19.053358  -74.12665   -14.682648  -22.124979
   -36.599243  -48.74176   -63.276165 -105.931175  -55.73878   -29.641474
   -82.02079   -66.07086   -86.092316  -12.538182  -52.26112   -56.611908
   -53.72919   -68.04334   -71.75747   -25.230314  -59.705933  -59.913013
   -80.58071   -70.01344   -76.735435  -57.225006  -99.50271   -64.93602
   -87.31793   -41.185997 -111.82865    96.7603    -25.541542  -47.19406
     9.682261  100.54758   -18.957794  -65.14736   -53.31745   115.89349
   -37.69542   -29.678658   68.49148   -21.12757   -58.475235  115.771095
   -48.918766  -10.810768    9.442081  -25.931438   -4.91963    69.91606
    -8.367846   -9.829544  -29.277523   37.668896  -81.35674 ]]
[[ -73.18694   -104.18346    -79.67243    -69.5926     -32.331043
   -26.919044   -67.92018    -70.269844   -54.916836   -41.552197
   -77.832436   -68.71284    -51.24168    -78.12334    -78.703705
  

[[  68.85753     -34.64262     -64.37783     -76.92999    -112.09228
    28.249151    -90.08788     -21.08038     -72.65131     -16.84379
   -57.931583    -61.16381     -43.712578     45.593716     52.31186
    34.039345     17.8969        6.290326     57.393776     33.392326
    48.370304     51.80203       8.439166     -3.9342465    19.863813
    44.34668      41.516766     30.072134     27.419168    -13.555752
    31.89911      39.74596      63.06704      11.406101    -11.700692
    63.216152    -72.10583      36.736385    -38.947784     -5.8593493
    -4.3851295   -23.93957     -24.898228     -9.286165     -1.3789754
   -23.261618     -8.716603     -0.85269487  -32.18698     -36.059444
    -8.581244    -14.668399    -22.800877     -1.4547443   -11.567736
   -48.326176    -26.979221      8.828254      5.126587    -39.915703
   -13.139912     12.400107    -50.686348     -0.9757239   -33.954765  ]]
[[ 2.1066574e+01 -7.1962486e+01 -8.2141075e+01 -3.7367493e+01
  -5.5121883e+01 -3.31996

[[  19.344912    53.98296     -3.3686247 -100.42178    -87.29461
    47.511196    32.95891    -27.436117    19.837423   -88.020164
    -0.8921636    6.0911484   11.379193   -91.38418    -77.39525
   -47.220715   -90.164116   -46.946568   -51.051273   -36.8913
   -71.03864    -76.975105   -58.458138   -44.25223    -65.16599
   -67.528564   -28.234539   -91.2912     -47.136578   -62.950314
   -46.176582   -65.13321    -67.40682    -76.53083    -78.967705
   -59.578438   -82.059074  -100.80404    -68.54545     12.451002
   -10.167402    13.933501     8.423928    29.249907    -2.8727582
     6.6093426  -32.313606    22.04445     -7.7013783    2.4515495
    23.155163    22.257195   105.70164     11.074082     7.1231112
   -44.954876    21.885872    33.182514    40.910225    48.87131
    11.444987    37.327583   -17.637255    -9.91608     -0.176555 ]]
[[ 2.0858793e+00  3.3813034e+01 -9.1366568e+00 -8.9571732e+01
  -1.0634267e+02  2.9602707e+01  3.1183084e+01  1.6955341e+01
   1.0741625e+01 -

[[ 5.20901756e+01  6.76884537e+01  3.06284542e+01 -5.70511627e+01
  -5.69577980e+01 -9.08196068e+00  6.01388969e+01  2.63485184e+01
   5.90198975e+01 -4.26454239e+01  4.85896072e+01  4.76501274e+01
   4.18601456e+01 -8.21390152e+01 -4.42996063e+01 -5.11080246e+01
  -1.03298584e+02 -1.13263626e+02 -6.21243629e+01 -5.62673454e+01
  -8.27220688e+01 -8.71064911e+01 -5.78783684e+01 -5.77573738e+01
  -3.66386223e+01 -3.12305489e+01 -4.75662384e+01 -9.25801926e+01
  -6.45246277e+01 -6.38183784e+01 -3.84407806e+01 -7.28281097e+01
  -6.40197449e+01 -8.44706345e+01 -6.16854477e+01 -7.53505249e+01
  -5.47946892e+01 -7.03222351e+01 -6.50975647e+01 -6.76133919e+00
   2.25359702e+00  8.09861565e+00 -3.17221189e+00 -4.19981480e+00
   3.33632545e+01  1.72637808e+00 -5.68569069e+01  9.65772057e+00
  -6.89173937e+00 -1.29348698e+01  3.20232315e+01  4.68156776e+01
   3.11426926e+01  2.63837051e+01 -5.59370565e+00 -3.16502609e+01
   2.12218189e+01  1.00512952e-01  7.22971916e+00  5.51039391e+01
   9.64786

In [70]:
if not train_flag:
    print(generate_text(model, start_string=u"ROMEO: "))


[[-3.2007744e+00 -1.6192927e+00 -1.6421026e+00  2.6736126e+00
   2.7076116e+00 -1.3007604e+00 -1.8109664e+00 -2.7480578e+00
  -1.2816085e+00  1.8285506e+00  2.6392164e+00 -1.7535639e+00
  -1.9200523e+00  6.3119249e+00  2.6279104e+00  5.9952860e+00
   7.1121106e+00  4.8515143e+00  1.0423810e+00  5.3729906e+00
   1.4373643e+00  6.8225203e+00  2.1744745e+00  6.3180432e+00
   3.7406433e+00  3.1416855e+00  2.8272407e+00  4.9923239e+00
   1.0993068e+00  1.9590818e+00  4.2241025e-01  3.3066869e+00
   3.5802681e+00  5.9890871e+00  2.6813745e+00  4.5076380e+00
   3.0891430e+00  6.1280947e+00  4.3073115e+00 -1.1171166e+00
  -2.7375004e+00 -3.8181505e-01 -2.1261780e-02 -1.9281676e-01
  -2.2451856e+00 -1.5364711e+00 -9.5868909e-01 -4.1836637e-01
   3.3471817e-01  1.4704579e+00 -3.5076861e+00 -2.9344704e+00
  -5.0054483e+00 -5.9737551e-01 -3.5360613e+00  1.2162752e-01
  -7.5104327e+00 -2.7082500e+00 -2.6150811e+00 -1.6968920e+00
  -1.6243312e+00 -2.9579504e+00 -1.4746709e+00  1.7756149e-03
   5.008

[[ 8.448035    8.258477    6.556043   -3.2427757  -6.385843    5.4427366
   7.5775557   3.723022    9.169636   -2.6588693   6.538093    6.7634263
   7.0597916  -3.4007394  -3.8945215  -3.1828988  -3.6418674  -5.2519727
  -5.7830124  -4.0504184  -3.8594227  -4.948115   -4.670886   -4.615652
  -5.3125343  -4.7755795  -5.970751   -4.319981   -2.1000557  -4.6951337
  -7.516309   -5.3943353  -2.5864716  -4.555641   -6.5664887  -3.4077463
  -6.8343167  -6.623809   -1.0974921   2.148852   -1.1205264  -2.0363214
  -5.414128    2.14238    -1.0664707  -4.6229897  -0.323118    1.9603719
  -5.067966   -3.3868985   1.5142627  -0.77190465 -3.4436862   4.00658
   0.71739876 -2.0492938  -7.131145   -0.28683013  0.7976793   0.6743134
  -4.7065177  -2.3484201  -6.998974    1.5205244  -4.595467  ]]
[[-7.1035633  -8.9763     -5.841923   -5.9733973  -6.4982014   0.79124767
  -7.651041   -5.8226275  -6.39821    -6.580386   -6.7392926  -6.714578
  -6.2837167  -0.38939267 -0.26139644  1.2967594  -0.05966178 -

[[ 6.4946585e+00  7.7929969e+00  4.6468544e+00 -5.2070322e+00
  -8.8681479e+00  1.5317886e+00  6.8228683e+00  2.3023918e+00
   4.8321452e+00 -3.4511123e+00  2.8710897e+00  5.2123675e+00
   2.5600004e+00 -2.9611001e+00 -4.3385983e+00 -2.5009673e+00
  -5.2028441e+00 -5.3831830e+00 -4.2819128e+00 -3.8331037e+00
  -5.4537263e+00 -4.3566256e+00 -5.1450753e+00 -1.5377679e+00
  -5.7775555e+00 -4.3931336e+00 -3.3084998e+00 -2.1723020e+00
  -2.6634669e+00 -4.8200035e+00 -9.5877037e+00 -4.7100549e+00
  -3.0949287e+00 -5.5422435e+00 -4.2016449e+00 -2.2371347e+00
  -3.2590971e+00 -7.7705851e+00 -1.6290910e+00  1.9749089e+00
  -8.9244717e-01 -1.2735126e+00 -4.7428241e+00  2.0625379e+00
   6.6567665e-01 -1.6758943e+00  1.6402498e+00  7.0121688e-01
  -4.8031201e+00 -8.5441852e-01 -3.9697923e-03  9.8763186e-01
  -2.6837258e+00  1.2907610e+00 -4.0783399e-01 -2.2055984e+00
  -6.7991333e+00  1.0107274e+00  2.1782277e+00  6.9409817e-01
  -4.5754080e+00  8.5504252e-01 -6.1203957e+00 -1.8757354e+00
  -4.974

[[-1.0456398e+01 -6.8803139e+00 -6.9253826e+00 -8.1315327e+00
  -6.8527451e+00  4.0562444e+00 -8.4350309e+00 -4.8333392e+00
  -9.4759817e+00 -9.3481741e+00 -9.5442295e+00 -6.7061906e+00
  -9.9158640e+00 -7.7798694e-01  9.2057690e-02  1.5984551e+00
  -2.0034115e+00  5.5152684e-01  3.8876384e-03  2.3042603e+00
  -4.8530996e-01  3.3017895e+00 -8.6167502e-01 -7.1563399e-01
   1.0633472e+00  9.9630034e-01 -1.7528099e-01  6.0180300e-01
   2.4050405e+00 -1.5555519e+00  3.6905909e-01 -1.1368766e+00
   1.1573651e+00 -3.3633530e+00 -5.3959298e-01 -9.0614066e-02
  -5.8920302e+00 -3.1260312e-01 -7.3796067e+00  4.0863338e+00
   3.3394554e+00  1.5879457e+00  1.8154024e+00  7.0013684e-01
   2.4685187e+00  1.4902629e+00  3.2050614e+00  2.4474845e+00
   7.8300141e-02 -4.4617298e-01  2.8012650e+00  4.8473706e+00
   1.4542048e+00  3.8141041e+00  2.0727777e+00 -2.4286344e+00
   8.7377220e-01  3.2905884e+00  4.6548810e+00  9.4438678e-01
  -8.7417382e-01  3.4953341e+00 -5.2995877e+00  2.5223615e+00
  -2.949

[[-4.809596   -0.28884026 -4.7686434  -7.4151216  -4.0814605  -5.734241
  -3.1640058  -7.2109537  -4.469926   -3.5365753  -0.3696763  -4.3138623
  -5.065073    4.8402624   1.095615   11.621108    4.316151    4.431632
  -0.5603818   4.224665    0.52085227  9.288118   -3.9271903   2.6689863
   3.036311    4.113203    1.5878174  -1.0367076  -1.3156679  -0.89928055
  -1.8727595   4.4278154   5.6026864   3.3710139  -1.570725    2.5154147
  -2.1875937   3.288457   -5.315209    0.44122565  1.2194614  -0.88649946
  -1.2606211  -2.580276   -3.52133    -2.1623862  -3.0278413   2.0271006
  -1.6549747   0.5399983  -2.7244744  -2.421044   -2.7352815   1.5021778
  -3.9119444   0.40063998 -5.8781133   0.6433084   1.1928426  -1.2528229
  -5.311374   -3.3548622  -8.678455   -1.2738756  -1.4554908 ]]
[[-7.7807913  -1.0017086  -4.961321   -3.9777138  -2.6291442  -5.975852
  -3.077798   -5.284153   -3.0296123  -6.182995   -2.4741285  -2.7564988
  -5.1424065   7.286941   -0.36856025  1.3565395  -2.2691963 

...

[[-1.3733562   3.4142118  -1.5082006  -3.7687206  -3.9034412  -1.3931448
  -0.19903935 -1.2927663  -2.2364507  -4.0951986  -0.12527716 -2.841749
  -2.125454    0.06895983  5.372157   -3.185287    3.6990104   5.4294214
  -0.9076179  -0.20538977  2.291484    2.7055106  -0.9877049  -0.8086822
   4.153213    2.3344343   7.028478   -0.36817682  2.0825768   0.9567267
   5.786754    2.2243133   1.0669134   4.986657   -2.455913   -1.519592
   0.37880456  1.041555   -4.4929194  -3.960439    2.0014133   1.051527
   1.1138449  -3.9300172   1.733851    0.05513194  0.6224965  -4.154013
  -0.0254666  -1.3111867   4.9934616   0.31649184  2.58848    -6.700518
   2.9975195  -1.0430545   0.8736691   1.3216015  -0.6236771  -1.6246995
  -2.5874841   1.0521944  -1.4465758  -1.277543   -3.4105446 ]]
[[-3.7400625e+00  2.2304783e+00 -1.6781791e+00 -6.4662447e+00
  -4.6024694e+00 -5.0396533e+00  8.6058146e-01 -2.9232152e+00
  -7.8972447e-01 -6.6271844e+00 -2.7641675e+00 -2.9453781e+00
  -3.2516549e+00 -3.99309

[[ -8.177306    -3.9158947   -4.9139194   -7.4804225  -10.091306
   -4.662236    -1.9670682   -4.012774    -3.5766225   -7.769871
   -5.0563154   -3.8468733   -3.6636012   -4.282073    -6.2782464
   -3.0511286   -5.9378066   -3.0819273   -2.2718313   -6.8228292
   -0.7085664   -3.5689502   -4.287962    -3.8839133   -3.363513
   -6.3930316  -11.26633     -6.1063194   -2.7844439   -5.1112633
   -6.5355296   -7.0151443   -4.855828    -2.914471    -7.71821
   -4.3611116   -4.5601134   -7.114222    -8.04688      5.299862
    0.3706553    3.220663    -6.0704913    4.729718     0.03732113
   -1.3668838    6.863771     5.3704205   -2.4565225    5.308681
    5.8036666    1.1137295   -1.7248669    3.0443192   -1.6660783
   -0.31944084   2.98833     -0.26734388  11.051998     7.6947865
   -2.227691    -5.6880865   -5.566595     4.0775595    0.15709913]]
[[  3.4676845   8.83609     0.6431136  -9.521575  -13.283631    2.7421844
    5.44179     1.0915791   5.264519   -5.2982917   2.4082153   3.03230

[[  3.755137     8.42618      1.5767652   -7.848279   -13.153535
    6.082187     4.8196616    1.1282269    3.5057354   -5.543649
    0.48073515   1.8121635   -0.49973553  -0.23091602  -5.393832
   -5.321953    -8.576415    -4.1739864   -4.7512884   -6.773422
   -6.5252986   -2.575355    -5.139233    -2.7955072   -6.327412
   -4.2130694   -6.9186463   -3.8328102   -4.921497    -6.030652
   -8.321846    -3.834198    -4.0674777   -5.6587777   -5.294793
   -6.00177     -7.3093452   -7.595314    -7.3090196    3.2952218
   -1.3180346   -0.08635969  -1.9755877    4.7179427    0.71328634
   -3.8433602    2.3528736    4.437051    -6.127516    -4.601039
    2.7414663    1.7800528   -1.3530767    4.3966722   -0.7329645
   -2.8443847   -5.3009467   -0.4057659    1.5646158    1.4071589
   -5.931146    -0.11158418  -6.932378    -0.36798283  -8.427802  ]]
[[-10.948225    -8.726895    -9.32614     -7.5068874  -10.917118
    2.028611    -9.614235    -7.999902    -9.68035     -8.175322
   -9.356445    

[[  0.20703173   1.8085349   -1.7205822  -10.84138    -10.410069
    2.2721653    0.09698509  -4.1842523   -0.71328634  -8.235428
   -0.5586146   -2.4403565   -1.3529359   -3.3138716   -6.9032063
  -10.322889   -12.215231    -1.5572865   -5.0274715   -7.3351707
   -9.786421    -0.812108    -7.5672045  -12.408491   -14.32181
   -8.482441    -8.748608    -4.309041    -8.076151    -7.1329226
   -6.6890006  -10.427769   -10.412548    -5.6115737   -6.1448445
   -8.15025    -13.544629    -7.93828     -7.1297917   13.994317
   -0.34474054  -2.2924285   -7.672507     9.822843    -1.3397095
   -6.9739866   -1.5734142    9.749909    -3.674198    -5.417227
    1.1267452    0.8221438    0.86193454   9.652457    -2.0690682
   -1.7277397    3.5594392   -0.3200478   -0.24060242   7.9497313
   -6.6769595    1.4055345   -4.9217663    3.4736514   -5.9018564 ]]
[[-1.0535448  -1.2171705   0.3818515  -5.8233657  -9.3340225   2.7142835
  -1.3838054  -2.1628523  -3.018244   -6.581894   -3.314221   -2.7506144

[[ -0.47878245   3.0584867   -3.9495566  -10.216097   -10.337214
    1.450981    -0.08880726   1.0229412   -1.0719548   -5.9164357
   -2.3048818   -0.7843067   -1.97799     -3.465258    -3.3260002
   -2.5838225   -2.549152    -2.8179708   -4.186246    -4.9793477
   -3.158983    -3.097713    -6.5359077   -4.703771    -7.6421795
   -5.2274337   -2.4586709   -3.002838    -1.6043553   -7.2868834
   -7.407603    -2.1654973   -2.5664887   -3.6156497   -5.2030573
   -2.9663854   -8.966295    -4.4425673   -6.1954746    0.29903162
    1.5477508    0.9761548    0.38203207   8.878627    -1.7796226
   -2.0275912    4.202563     0.84343773  -5.9301996    1.671994
   -1.8339232    1.4789964   -2.0862648    2.2388198    3.5990486
   -3.1788044   -8.933117     3.8738823    3.3055649    2.023761
   -3.9585142   -2.1378102   -9.609791    -1.5960739   -5.771205  ]]
ROMEO: this took out of pleasuness father.
The jein the gods.

GLOUCES:
Go, dispatch, my kinsman take arms.

MARCIUS:
Sir, I centurel feel.



[[1 0 1 0 0]]
